In [63]:
import csv
from math import log

import pandas as pd

# Read CSV data
stocks = pd.read_csv("Stock_Market_New_Version.csv")
# print(stocks.keys()) # have 41 keys, 1st is "Date", 2nd is "SP500"
all_countries = stocks.keys()[2:]
num_dates = len(stocks["Date"])

#---------- a ----------
# crr: compounded return rate
all_lags = ["lag0", "lag2", "lag3", "lag4", "lag5", "lag6", "lag20", "lag40", "lag60"]
crr = dict.fromkeys(all_countries, None)
for country in all_countries:
    crr[country] = dict.fromkeys(all_lags)
    
    # process ln(stock) for each country for each lag.
    for lag in all_lags:

        t = int(lag[3:])

        print('Compute for Country(%s) lag(%s)  ' % (country, lag))
        #print('%d: %g' % (i, log(stocks[country][i]) - log(stocks[country][i-lag])))
        crr[country][lag] = ['']*num_dates

        for i in range(t, num_dates):
            if stocks[country][i] == 0.0 or stocks[country][i-t] == 0.0:
                continue
            elif t == 0:
                try:
                    crr[country][lag][i] = log(stocks[country][i]) -log(stocks[country][i-1])
                except:
                    crr[country][lag][i] = ""
            else:
                crr[country][lag][i] = log(stocks[country][i]) - log(stocks[country][i-t])

#print(crr["Canada"]["lag0"][0:5])
#---------- b ----------
violatility = dict.fromkeys(all_countries, None)
for country in all_countries:
    violatility[country] = dict.fromkeys(all_lags)

    # process ln(stock) for each country for each lag.
    for lag in all_lags:
        violatility[country][lag] = [''] *num_dates
        for i, val in enumerate(crr[country][lag]):
            if val != '':
                violatility[country][lag][i] = val*val

#print(violatility["China"]["lag0"][650:700])

#---------- c ----------
crisis = dict.fromkeys(all_countries, None)
for country in all_countries:
    crisis[country] = [0] * num_dates
    for i in range(num_dates):

        # if there is no valid crr for this date of this country
        if crr[country]["lag0"][i] == '':
            crisis[country][i] = 0
            continue

        if i+501 >= num_dates:
            samples = crr[country]["lag0"][(num_dates-501):num_dates]
        else:
            samples = crr[country]["lag0"][i:i+501]

        # 0.6% is located at 3th.
        # 0 -> 1/500 -> 2/500 -> .... -> 10/200
        samples.sort()
        if crr[country]["lag0"][i] < samples[3]:
            crisis[country][i] = 1
    
#print(crisis[country][650:700])
# usage: crisis["China"] will return a Python list of value.

#---------- d ----------
regions = {}
regions["Asia"] = [6, "China", "Japan", "India", "Korea", "Indonesia",
        "Taiwan", "Thailand", "Hong_Kong", "Philippines", 
        "Malaysia", "Pakistan", "Australian", "New Zealand"]

regions["Europe"] = [8, "Germany", "England", "France", "Russia", "Spain",
          "Netherlands", "Switzerland", "Sweden", "Poland", 
          "Belgium", "Austria", "Norway", "Israel", "Denmark", 
          "Ireland", "Greece", "Czech_Republic", "Hungary", "Slovakia"]

regions["America"] = [3, "Canada", "Argentina", "Peru", "Brazil", "Chile", "Mexico", "America"]

region_crisis = dict.fromkeys(regions.keys())

for region in regions.keys():
    region_crisis[region] = [0]*num_dates

    for i in range(num_dates):
        count = 0
        for country in regions[region][1:]:
            count += crisis[country][i]
        
        if count >= regions[region][0]:
            region_crisis[region][i] = 1

# usage: region_crisis["Asia"] will return a Python of value.

#---------- e ----------
global_crisis = [0]*num_dates

for i in range(num_dates):
    count = 0
    for region in region_crisis.keys():
        count += region_crisis[region][i]
    if count >= 2:
        global_crisis[i] = 1

with open("global_crisis.csv", mode="w", newline='') as f:
    w = csv.writer(f)
    w.writerow(['Date', 'Global Crisis'])

    for i in range(num_dates):
        w.writerow([stocks["Date"][i], global_crisis[i]])

#---------- f ----------
global_crisis_past20 = [0]*num_dates
for i in range(num_dates):
    if sum(global_crisis[i:i+21]):
        global_crisis_past20[i] = 1

with open("global_crisis_past20.csv", mode="w", newline='') as f:
    w = csv.writer(f)
    w.writerow(['Date', 'Global Crisis 20'])

    for i in range(num_dates):
        w.writerow([stocks["Date"][i], global_crisis_past20[i]])

#---------- g ----------
# [0,1,2,3,4] = 5 days.
# that's why use i-4 and i+1
avg_country_crisis_past5 = dict.fromkeys(all_countries, None)
for country in all_countries:
    avg_country_crisis_past5[country] = [''] * num_dates
    for i in range(5-1, num_dates): 
        avg_country_crisis_past5[country][i] = sum(crisis[country][(i-4):(i+1)]) / min(5, num_dates-i)

#---------- h ----------
avg_country_crisis_past20 = dict.fromkeys(all_countries, None)
for country in all_countries:
    avg_country_crisis_past20[country] = [''] * num_dates
    for i in range(20-1, num_dates): 
        avg_country_crisis_past20[country][i] = sum(crisis[country][(i-19):(i+1)]) / min(20, num_dates-i)

#---------- i ----------
avg_region_crisis_past5 = dict.fromkeys(regions.keys(), None)
for region in regions.keys():
    avg_region_crisis_past5[region] = [''] * num_dates
    for i in range(5-1, num_dates): 
        avg_region_crisis_past5[region][i] = sum(region_crisis[region][(i-4):(i+1)]) / min(5, num_dates-i)

#---------- j ----------
avg_region_crisis_past20 = dict.fromkeys(regions.keys(), None)
for region in regions.keys():
    avg_region_crisis_past20[region] = [''] * num_dates
    for i in range(20-1, num_dates): 
        avg_region_crisis_past20[region][i] = sum(region_crisis[region][(i-19):(i+1)]) / min(20, num_dates-i)

#---------- k ----------
avg_global_crisis_past5 = [''] * num_dates
for i in range(5-1, num_dates): 
    avg_global_crisis_past5[i] = sum(global_crisis[(i-4):(i+1)]) / min(5, num_dates-i)
    
#---------- l ----------
avg_global_crisis_past20 = [''] * num_dates
for i in range(20-1, num_dates): 
    avg_global_crisis_past20[i] = sum(global_crisis[(i-19):(i+1)]) / min(20, num_dates-i)

#---------- Exportation ----------
with open("stock_results.csv", mode="w", newline='') as f:
    w = csv.writer(f)

    header = ['Date']

    # header for a.
    for country in all_countries:
        for lag in all_lags:
            header.append("crr_%s_%s" % (country, lag))
    # header for b.
    for country in all_countries:
        for lag in all_lags:
            header.append("volatility_%s_%s" % (country, lag))
    # header for c.
    for country in all_countries:
        header.append("crisis_%s" % country)
    # header for d.
    for region in regions.keys():
        header.append("region_crisis_%s" % region)
    # header for g.
    for country in all_countries:
        header.append("average_crisis_past5_%s" % country)
    # header for h.
    for country in all_countries:
        header.append("average_crisis_past20_%s" % country)
    # header for i.
    for region in regions.keys():
        header.append("average_crisis_past5_%s" % region)
    # header for j.
    for region in regions.keys():
        header.append("average_crisis_past20_%s" % region)
    # header for k.
    header.append("average_global_crisis_past5")
    # header for l.
    header.append("average_global_crisis_past20")

    w.writerow(header)

    for i in range(num_dates):
        data = [stocks["Date"][i]]

        # data for a.
        for country in all_countries:
            for lag in all_lags:
                data.append(crr[country][lag][i])
        # data for b.
        for country in all_countries:
            for lag in all_lags:
                data.append(violatility[country][lag][i])
        # data for c.
        for country in all_countries:
            data.append(crisis[country][i])
        # data for d.
        for region in regions.keys():
            data.append(region_crisis[region][i])
        # data for g.
        for country in all_countries:
            data.append(avg_country_crisis_past5[country][i])
        # data for h.
        for country in all_countries:
            data.append(avg_country_crisis_past20[country][i])
        # data for i.
        for region in regions.keys():
            data.append(avg_region_crisis_past5[region][i])
        # data for j.
        for region in regions.keys():
            data.append(avg_region_crisis_past20[region][i])
        # data for k.
        data.append(avg_global_crisis_past5[i])
        # data for l.
        data.append(avg_global_crisis_past20[i])

        w.writerow(data)

Compute for Country(Canada) lag(lag0)  
Compute for Country(Canada) lag(lag2)  
Compute for Country(Canada) lag(lag3)  
Compute for Country(Canada) lag(lag4)  
Compute for Country(Canada) lag(lag5)  
Compute for Country(Canada) lag(lag6)  
Compute for Country(Canada) lag(lag20)  
Compute for Country(Canada) lag(lag40)  
Compute for Country(Canada) lag(lag60)  
Compute for Country(Argentina) lag(lag0)  
Compute for Country(Argentina) lag(lag2)  
Compute for Country(Argentina) lag(lag3)  
Compute for Country(Argentina) lag(lag4)  
Compute for Country(Argentina) lag(lag5)  
Compute for Country(Argentina) lag(lag6)  
Compute for Country(Argentina) lag(lag20)  
Compute for Country(Argentina) lag(lag40)  
Compute for Country(Argentina) lag(lag60)  
Compute for Country(Peru) lag(lag0)  
Compute for Country(Peru) lag(lag2)  
Compute for Country(Peru) lag(lag3)  
Compute for Country(Peru) lag(lag4)  
Compute for Country(Peru) lag(lag5)  
Compute for Country(Peru) lag(lag6)  
Compute for Country

Compute for Country(Czech_Republic) lag(lag4)  
Compute for Country(Czech_Republic) lag(lag5)  
Compute for Country(Czech_Republic) lag(lag6)  
Compute for Country(Czech_Republic) lag(lag20)  
Compute for Country(Czech_Republic) lag(lag40)  
Compute for Country(Czech_Republic) lag(lag60)  
Compute for Country(Hungary) lag(lag0)  
Compute for Country(Hungary) lag(lag2)  
Compute for Country(Hungary) lag(lag3)  
Compute for Country(Hungary) lag(lag4)  
Compute for Country(Hungary) lag(lag5)  
Compute for Country(Hungary) lag(lag6)  
Compute for Country(Hungary) lag(lag20)  
Compute for Country(Hungary) lag(lag40)  
Compute for Country(Hungary) lag(lag60)  
Compute for Country(Slovakia) lag(lag0)  
Compute for Country(Slovakia) lag(lag2)  
Compute for Country(Slovakia) lag(lag3)  
Compute for Country(Slovakia) lag(lag4)  
Compute for Country(Slovakia) lag(lag5)  
Compute for Country(Slovakia) lag(lag6)  
Compute for Country(Slovakia) lag(lag20)  
Compute for Country(Slovakia) lag(lag40)  

In [64]:
import pandas as pd
# merge my data and znn data
a = pd.read_csv("stock_results.csv")
b = pd.read_csv("bond_ad_er_all_data.csv")
merged = a.merge(b, on = 'Date')
merged.to_csv("explainable_variable.csv", index=False)